In [ ]:
# 
# data is at /projects/niblab/experiments/bbx/data/fmriprep
# test dict is at: /projects/niblab/data/BBX_gtest/
import glob
import os
import subprocess
import argparse
def startup(arglist, datapath, basepath):
	sub = arglist['SUB']
	ses = arglist['SES']
	dat = os.path.join(basepath, sub, ses)
	print(dat)
#	Make the directory if needed
	if os.path.exists(dat) == True:
		print('dictionary exists for %s %s'%(sub, ses))	
	else:
		print('doesnt exist')
		os.makedirs(dat)
		os.makedirs(os.path.join(dat, 'func'))
		os.makedirs(os.path.join(dat, 'anat'))
def brainX(datapath, sub, ses, basepath):
#	Start with skull strip 
	iput = os.path.join(datapath, sub, ses, 'func', '%s_%s_task-training_run-*_bold.nii.gz'%(sub, ses))
	for run in glob.glob(iput): 
		oput0 = run.split('/')[-1].split('.')[0]
		oput = os.path.join(basepath,sub,ses,'func', oput0)
		bet_cmd = ['bet' ,'%s'%run,'%s_brain.nii.gz'%oput, '-F']
		print(bet_cmd)
		bet_process=subprocess.Popen(bet_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
		stdout, stderr = bet_process.communicate()
		bet_process.wait()
	print('brain extraction done')
def MOCO(basepath, sub, ses):
	inpt = os.path.join(basepath,sub,ses,'func', '%s_%s_task-training_run-*_bold_brain.nii.gz'%(sub, ses))
	txtfile = os.path.join(basepath, sub, ses, 'func', 'outliers_output.txt')
	for run in glob.glob(inpt):
		otpt0 = run.split('.')[0]
		otpt = '%s_confounds.txt'%otpt0
		plotpt = '%s_moco.png'%otpt0
		#moco_cmd = ['fsl_motion_outliers', '-i', run, '-o', otpt, '-s',filept ,'-p', plotpt, '--fd', '--thresh=0.9']
		moco_cmd = ['fsl_motion_outliers', '-i', run, '-o', otpt, '-p', plotpt, '--fd', '--thresh=0.9', '-v']
		print(moco_cmd)
		with open(txtfile, 'w') as txtf:
			moco_process=subprocess.Popen(moco_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
			stdout, stderr = moco_process.communicate()
			moco_process.wait()
			txtf.write(stdout)
		txtf.close()
def main(arglist):
	basepath = '/projects/niblab/data/BBX_gtest/data/pseudoprep'
	datapath = '/projects/niblab/experiments/bbx/data/bids'
	startup(arglist, datapath, basepath)
	if arglist['BRAINX'] == True:
		brainX(datapath, arglist['SUB'],arglist['SES'], basepath)
	if arglist['MOCO'] == True:
		MOCO(basepath, arglist['SUB'],arglist['SES'])
if __name__ == "__main__":
   #commandline parser
   parser=argparse.ArgumentParser(description='Its for the streetz')
   parser.add_argument('-session', dest='SES', action='store',
			default=False, help='which session are we running on?')
   parser.add_argument('-subject', dest='SUB', action='store',
			default=False, help='which subject to analyze')
   parser.add_argument('-brainX', dest='BRAINX', action='store_true',
                       default=False, help='BET skull strip')
   parser.add_argument('-moco', dest='MOCO', action='store_true',
			default=False, help='fsl outliers motion correction')
   args = parser.parse_args()
   arglist={}
   for a in args._get_kwargs():
       arglist[a[0]]=a[1]
   main(arglist)
	